In [2]:
import math
import numpy as np
import random
from random import choices
from typing import TypeVar,List
from sortedcontainers import SortedSet
import heapq
import numpy as np
import matplotlib.pyplot as plt
import collections
from colorama import init, Fore

In [77]:
class cell:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        self.b = False
        self.g = 0
        self.f = 0
        self.h = 0
        self.h_new = 0
        self.pointer = None
    def __lt__(self, other: cell) -> bool:
        if((self.f) != (other.f)):
            return (self.f) < (other.f)
        else:
            return self.g < other.g
    def set_hval(self,t_x,t_y):
        self.h = abs(t_x - self.x) + abs(t_y - self.y)
        self.f = self.g + self.h
    def set_fvalAdap(self):
        self.f = self.h + self.g

In [7]:
T = TypeVar('T')
c = cell(1,2)
c.set_hval(8,9)
print(c.h, c.f)
x = cell(7,9)
li = list()
#heapq.heapify(li)
heapq.heappush(li,x)
y = cell(6,7)
heapq.heappush(li,y)
li.remove(x)
z = heapq.heappop(li)
print(z.x)


14 14
6


In [8]:
class maze:
    def __init__(self,rows,columns,sx,sy,tx,ty,probability):
        self.rows = rows
        self.columns = columns
        self.sx = sx
        self.sy = sy
        self.tx = tx
        self.ty = ty
        self.visited = [[False for i in range(self.rows)] for j in range(self.columns)]
        self.options = ['b','u']
        self.weights = [probability,1-probability]
        self.drow = [0, 1, 0, -1]
        self.dcol = [-1, 0, 1, 0]
        #p = choices(self.options,self.weights)
        #print(p)
        self.mz = [] 
        for i in range(self.rows):
            arr = []
            for j in range(self.columns):
                c = cell(i,j)
                arr.append(c)
            self.mz.append(arr)            
    def isValid(self,r,c):
        if (r < 0 or c < 0 or r >= self.rows or c >= self.columns):
            return False
        if (self.visited[r][c]):
            return False
        return True
    def DFS(self,grid,sx,sy,tx,ty):
        stk = []
        stk.append([sx, sy])
        while(len(stk)>0):
            curr = stk[len(stk) - 1]
            stk.remove(stk[len(stk) - 1])
            r = curr[0]
            c = curr[1]
            if(self.isValid(r, c) == False):
                continue
            p = choices(self.options,self.weights)
            self.visited[r][c] = True
            if((r!= sx or c != sy) and (r!=tx or c != ty)):
                if(p[0] == 'b'):
                    self.mz[r][c].b = True
            for i in range(4):
                adjx = r + self.drow[i]
                adjy = c + self.dcol[i]
                stk.append([adjx, adjy])


In [13]:
m = maze(5,5,0,0,4,4,0.3)
m.DFS([],0,0,4,4)
l = []
for i in range(5):
    a = []
    for j in range(5):
        if(i == m.sx and j == m.sy):
            a.append('-')
            print(Fore.BLUE, f's', end="")
        elif(i == m.tx and j == m.ty):
            a.append('-')
            print(Fore.BLUE, f't', end="")
        elif m.mz[i][j].b:
            a.append('x')
            #print(i,j)
            print(Fore.RED, f'x', end="")
        else:
            a.append('-')
            print(Fore.GREEN, f'-', end="")
    l.append(a)
    print('\n')


 s x - x -

 - - x - -

 - x - - -

 - - - x -

 - x x - t



In [98]:
def Astar(grid,start_cell_ind,goal_cell_ind, weight = 1):
    cells_processed = 0 # number of cells popped from the fringe before reaching the goal.
    number_of_clashes = 0 # number of time we had to update priorities in the fringe.
    start_cell = grid[start_cell_ind[0]][start_cell_ind[1]] # start cell of our grid.
    goal_cell = grid[goal_cell_ind[0]][goal_cell_ind[1]] # goal cell of our grid
    #open_list = SortedSet() # fringe, we have used sorted set as our priority queue
    open_list = list()
    close_list = set() # contains cells already processed, have used set for this purpose (O(1) lookup on average)
    parent_dict = {} # stores parent child relationship, using dictionary for this
    visited = set() # child already visited, using set for this purpose (O(1) lookup on average)
    start_cell.g = 0 # Setting the g value of the start cell to 0
    # setting the h and f value of the start cell using the weight and heuristic passed in the function arguments.
    # weight is for q9
    start_cell.set_hval(goal_cell_ind[0], goal_cell_ind[1])
    #open_list.add((start_cell.f, start_cell.h, (start_cell.x, start_cell.y))) # adding the start cell to the priority queue
    heapq.heappush(open_list,start_cell) 
    #print(heapq.heappop(open_list).x)
    # In the queue we are pushing a tuple whose first coordinate is the f value, second coordinate is the h value 
    # then a tuple containing the location of the cell. The priority is set according to the f value in case of a 
    # tie h value is seen.
    visited.add((start_cell.x, start_cell.y)) # updating the visited set as we are pushing the node in priority queue
    while len(open_list)>0: # loop over the priority queue until it is not empty or we reach the goal node
        #cl = open_list.pop(0) # pop the node with the least priority (it is at index 0 in the sorted set)
        cl = heapq.heappop(open_list)
        #curr_cell = grid[cl[-1][0]][cl[-1][1]] # the last index in cl contains the x and y coordinates, 
        # storing the node popped from the queue in curr_cell.
        curr_cell = grid[cl.x][cl.y]
        cells_processed += 1 # incrementing the number of cells processed (since we just popped from the queue)
        close_list.add((curr_cell.x, curr_cell.y)) # adding the node to the closed list as it is about to be processed
        if curr_cell == goal_cell: 
            # If the curr cell is the goal cell return the path from the curr cell to goal cell by backtracking 
            # from goal cell to curr cell using the parent dictionary as well as all the nodes 
            # that have been processed uptil now.
            path = []
            curr = curr_cell
            print("printing g value for goal state")
            print(grid[curr.x][curr.y].g)
            while (curr.x, curr.y) != (start_cell_ind[0], start_cell_ind[1]):
                path.append([curr.x, curr.y])
                curr = parent_dict[(curr.x, curr.y)]
            path.append([curr.x,curr.y])
            
            return path[::-1], cells_processed, number_of_clashes
        children = [] # This list is used to store all the possible valid children of the cell 
        # currently being processed.
        for new_pos in [(0,1),(0,-1),(1,0),(-1,0)]: # 4 main compass directions, field of view also the agent can 
            # only move to these locations
            node_pos = (curr_cell.x+new_pos[0], curr_cell.y+new_pos[1]) # position of the child
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # if the child is not part of the grid continue
                continue
            if(grid[node_pos[0]][node_pos[1]].b):
                # if the child is blocked continue
                continue
            if((node_pos[0], node_pos[1]) in close_list):
                # if child is in close list continue
                continue
            if((node_pos[0], node_pos[1]) in visited):
                # if the child is already visited then compare the previous cost(f) to the new cost(f) and if the
                # new cost is lower update the priorityimport of the child in the priority queue, update the 
                # parent dictionary and update the g value of the node.
                if curr_cell.g + 1 + grid[node_pos[0]][node_pos[1]].h < grid[node_pos[0]][node_pos[1]].f:
                    number_of_clashes += 1 # increment the number of clashes by 1
                    # To update the priority we remove the node from the sorted set (O(logn)) and reinsert with 
                    # new priority O(log(n)). 
                    #open_list.remove((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    #grid[node_pos[0]][node_pos[1]].g = curr_cell.g+1
                    #grid[node_pos[0]][node_pos[1]].set_hval(goal_cell_ind[0], goal_cell_ind[1])
                    #open_list.add((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    removalCell = grid[node_pos[0]][node_pos[1]]
                    open_list.remove(removalCell)
                    removalCell.g = curr_cell.g+1
                    removalCell.set_hval(goal_cell_ind[0], goal_cell_ind[1])
                    heapq.heappush(open_list,removalCell)                                
                    parent_dict[(node_pos[0],node_pos[1])] = curr_cell # updating the parent dictionary
                    #parent_list.append(curr_cell)
                    #grid[node_pos[0]][node_pos[1]].pointer = curr_cell
                    #print(grid[node_pos[0]][node_pos[1]].pointer)
            else:
                # If the child is not visited update its g value, set its h and f value and add it to the 
                # children list after updating its parent.
                grid[node_pos[0]][node_pos[1]].g = curr_cell.g + 1
                grid[node_pos[0]][node_pos[1]].set_hval(goal_cell_ind[0], goal_cell_ind[1])
                children.append(grid[node_pos[0]][node_pos[1]])
                parent_dict[(node_pos[0],node_pos[1])] = curr_cell
                #parent_list.append(curr_cell)
                #grid[node_pos[0]][node_pos[1]].pointer = curr_cell
        if(children == []): # If no valid child continue
            continue
        for child in children: # else if valid children exist add them to the open list and update the visited set.
            #open_list.add((child.f,child.h, (child.x, child.y)))
            heapq.heappush(open_list,child)                                        
            visited.add((child.x, child.y))
    return [], 0, 0

In [15]:
path, c, f = Astar(m.mz,[0,0],[9,9])
print(path)

IndexError: list index out of range

In [101]:
def display_maze(m,start_cell_ind, goal_cell_ind):
    sx = start_cell_ind[0]
    sy = start_cell_ind[1]
    tx = goal_cell_ind[0]
    ty = goal_cell_ind[1]
    l = []
    for i in range(101):
        a = []
        for j in range(101):
            if(i == sx and j == sy):
                a.append(3)
                print(Fore.BLUE, f's', end="")
            elif(i == tx and j == ty):
                a.append(2)
                print(Fore.BLUE, f't', end="")
            elif m[i][j].b:
                a.append(1)
                #print(i,j)
                print(Fore.RED, f'x', end="")
            else:
                a.append(0)
                print(Fore.GREEN, f'-', end="")
        l.append(a)
        
        print('\n')

In [87]:
def final_path(path,agent_env,real_mz,final_mz,start_cell_ind, goal_cell_ind):
    i = 0
    for index in range(len(path)):
        i = i + 1
        x_ind = path[index][0]
        y_ind = path[index][1]
        agent_env[x_ind][y_ind].x = real_mz[x_ind][y_ind].x
        agent_env[x_ind][y_ind].y = real_mz[x_ind][y_ind].y
        agent_env[x_ind][y_ind].b = real_mz[x_ind][y_ind].b
        final_mz[x_ind][y_ind].x = real_mz[x_ind][y_ind].x
        final_mz[x_ind][y_ind].y = real_mz[x_ind][y_ind].y
        final_mz[x_ind][y_ind].b = real_mz[x_ind][y_ind].b
        print(agent_env[x_ind][y_ind].f)
        if(real_mz[x_ind][y_ind].b):
            print("printing g value in final path")
            #display_maze(real_mz,start_cell_ind, goal_cell_ind)
            #display_maze(agent_env,start_cell_ind, goal_cell_ind)
            return path[:index]
        for new_pos in [(0,1),(0,-1),(1,0),(-1,0)]:
            child_pos = (x_ind+new_pos[0], y_ind+new_pos[1])
            if(child_pos[0] > len(real_mz)-1 or child_pos[1] > len(real_mz)-1 or child_pos[0] < 0 or child_pos[1] < 0):
                continue
            agent_env[child_pos[0]][child_pos[1]].x = real_mz[child_pos[0]][child_pos[1]].x
            agent_env[child_pos[0]][child_pos[1]].y = real_mz[child_pos[0]][child_pos[1]].y
            agent_env[child_pos[0]][child_pos[1]].b = real_mz[child_pos[0]][child_pos[1]].b
            final_mz[child_pos[0]][child_pos[1]].x = real_mz[child_pos[0]][child_pos[1]].x
            final_mz[child_pos[0]][child_pos[1]].y = real_mz[child_pos[0]][child_pos[1]].y
            final_mz[child_pos[0]][child_pos[1]].b = real_mz[child_pos[0]][child_pos[1]].b
    #display_maze(real_mz,start_cell_ind, goal_cell_ind)
    #display_maze(agent_env,start_cell_ind, goal_cell_ind)
    return path

In [91]:
def repeated_forward_astar(agent_env,real_mz,final_mz,start_cell_ind, goal_cell_ind):
    paths = []
    counter = 0
    while(1):
        counter += 1
        print(counter)
        pathA,c,f = Astar(agent_env,start_cell_ind,goal_cell_ind)
        #print(pathA)
        if(pathA == []):
            print("I cannot reach the target")
            break
        else:
            p = final_path(pathA,agent_env,real_mz,final_mz,start_cell_ind, goal_cell_ind)
            if(p[-1] == goal_cell_ind):
                print("I reached the target")
                print("final")
                display_maze(final_mz,start_cell_ind,goal_cell_ind)
                print("-------")
                display_maze(agent_env,start_cell_ind,goal_cell_ind)
                return path
            else:
                start_cell_ind = p[-1]

In [102]:
real_mz = maze(101,101,0,0,100,100,0.3)
real_mz.DFS([],0,0,100,100)
display_maze(real_mz.mz,[0,0],[100,100])

 s - - - - x x x x - - - - - - - - - x - - - x - - x - - - - - x - x x x x x - - - - - - - x - - - - - x - x - - - - - - - - x - - x - - x - - x - - - x x x x - x x - - - - x x - - x x - - - - - - - - -

 x x - - - x - - x - x x - - - - - - - - - - - - x x - - - - x - - x - x - - - x - - - - - - x x x x - - - - - x - - - - - x - - - - - x - - - x - - x - - x - - x x x - x - x - x x - - - - x x - - x - -

 - - x - - x - - - - - x x - - - - - - - x - - - x - - x x - x - - - - - - x x x x - x - - x - x - - x x - - - - x x - - - - - - - - x x - - - - x x - x x x x - - - - x - - - - - - x x - x - - - - - x -

 - - x - - - - - - - - - - - - - - - x x - x - - - x - - - - - x - - - x - - - x x x x x x - - - - - x - x - x x - - - x x - - x - - - - x - x x - - - x - x - - - - - x - - x - - - - - - - x x x x - - x

 x x - - - x - - - x - - x - - x - - - - x - x - - x - - - - - - - - - - x - - x - - x - - - x - x x - - x - - x x - - x x x - - - - - - x - x - - - - - - - - - x - - - - - - - x - x -

In [103]:
agent_env = maze(101,101,0,0,100,100,0)
agent_env.DFS([],0,0,100,100)
#display_maze(agent_env.mz,[0,0],[100,100])
final_mz = maze(101,101,0,0,100,100,1)
final_mz.DFS([],0,0,100,100)
#display_maze(final_mz.mz,[0,0],[100,100])
path = repeated_forward_astar(agent_env.mz,real_mz.mz,final_mz.mz,[0,0],[100,100])
print(path)

1
printing g value for goal state
200
200
200
200
200
200
200
printing g value in final path
2
printing g value for goal state
196
196
196
196
printing g value in final path
3
printing g value for goal state
195
195
195
195
printing g value in final path
4
printing g value for goal state
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
194
printing g value in final path
5
printing g value for goal state
180
180
180
180
180
180
printing g value in final path
6
printing g value for goal state
181
177
179
181
181
181
181
printing g value in final path
7
printing g value for goal state
177
177
177
177
177
printing g value in final path
8
printing g value for goal state
175
175
175
175
175
175
175
175
printing g value in final path
9
printing g value for goal state
170
170
170
170
170
170
170
170
printing g value in final path
10
printing g value for goal state
165
165
165
165
165
165
165
165
165
printing g value in final path
11
printing g value for goal state
159
159
159
15

In [75]:
def AdaptiveAstar(grid,start_cell_ind,goal_cell_ind, weight = 1):
    cells_processed = 0 # number of cells popped from the fringe before reaching the goal.
    number_of_clashes = 0 # number of time we had to update priorities in the fringe.
    start_cell = grid[start_cell_ind[0]][start_cell_ind[1]] # start cell of our grid.
    goal_cell = grid[goal_cell_ind[0]][goal_cell_ind[1]] # goal cell of our grid
    #open_list = SortedSet() # fringe, we have used sorted set as our priority queue
    open_list = list()
    close_list = set() # contains cells already processed, have used set for this purpose (O(1) lookup on average)
    parent_dict = {} # stores parent child relationship, using dictionary for this
    visited = set() # child already visited, using set for this purpose (O(1) lookup on average)
    start_cell.g = 0 # Setting the g value of the start cell to 0
    # setting the h and f value of the start cell using the weight and heuristic passed in the function arguments.
    # weight is for q9
    start_cell.set_fvalAdap()
    #open_list.add((start_cell.f, start_cell.h, (start_cell.x, start_cell.y))) # adding the start cell to the priority queue
    heapq.heappush(open_list,start_cell) 
    #print(heapq.heappop(open_list).x)
    # In the queue we are pushing a tuple whose first coordinate is the f value, second coordinate is the h value 
    # then a tuple containing the location of the cell. The priority is set according to the f value in case of a 
    # tie h value is seen.
    visited.add((start_cell.x, start_cell.y)) # updating the visited set as we are pushing the node in priority queue
    while len(open_list)>0: # loop over the priority queue until it is not empty or we reach the goal node
        #cl = open_list.pop(0) # pop the node with the least priority (it is at index 0 in the sorted set)
        cl = heapq.heappop(open_list)
        #curr_cell = grid[cl[-1][0]][cl[-1][1]] # the last index in cl contains the x and y coordinates, 
        # storing the node popped from the queue in curr_cell.
        curr_cell = grid[cl.x][cl.y]
        cells_processed += 1 # incrementing the number of cells processed (since we just popped from the queue)
        close_list.add((curr_cell.x, curr_cell.y)) # adding the node to the closed list as it is about to be processed
        if curr_cell == goal_cell: 
            # If the curr cell is the goal cell return the path from the curr cell to goal cell by backtracking 
            # from goal cell to curr cell using the parent dictionary as well as all the nodes 
            # that have been processed uptil now.
            path = []
            curr = curr_cell
            print("printing g value for goal state")
            print(grid[curr.x][curr.y].g)
            gs = grid[curr.x][curr.y].g
            while (curr.x, curr.y) != (start_cell_ind[0], start_cell_ind[1]):
                path.append([curr.x, curr.y])
                curr = parent_dict[(curr.x, curr.y)]
            path.append([curr.x,curr.y])
            print("close start")
            for val in close_list:
                grid[val[0]][val[1]].h = gs - grid[val[0]][val[1]].g
            print("close end")
            return path[::-1], cells_processed, number_of_clashes
        children = [] # This list is used to store all the possible valid children of the cell 
        # currently being processed.
        for new_pos in [(0,1),(0,-1),(1,0),(-1,0)]: # 4 main compass directions, field of view also the agent can 
            # only move to these locations
            node_pos = (curr_cell.x+new_pos[0], curr_cell.y+new_pos[1]) # position of the child
            if(node_pos[0] > len(grid)-1 or node_pos[1] > len(grid)-1 or node_pos[0] < 0 or node_pos[1] < 0):
                # if the child is not part of the grid continue
                continue
            if(grid[node_pos[0]][node_pos[1]].b):
                # if the child is blocked continue
                continue
            if((node_pos[0], node_pos[1]) in close_list):
                # if child is in close list continue
                continue
            if((node_pos[0], node_pos[1]) in visited):
                # if the child is already visited then compare the previous cost(f) to the new cost(f) and if the
                # new cost is lower update the priorityimport of the child in the priority queue, update the 
                # parent dictionary and update the g value of the node.
                if curr_cell.g + 1 + grid[node_pos[0]][node_pos[1]].h < grid[node_pos[0]][node_pos[1]].f:
                    number_of_clashes += 1 # increment the number of clashes by 1
                    # To update the priority we remove the node from the sorted set (O(logn)) and reinsert with 
                    # new priority O(log(n)). 
                    #open_list.remove((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    #grid[node_pos[0]][node_pos[1]].g = curr_cell.g+1
                    #grid[node_pos[0]][node_pos[1]].set_hval(goal_cell_ind[0], goal_cell_ind[1])
                    #open_list.add((grid[node_pos[0]][node_pos[1]].f, grid[node_pos[0]][node_pos[1]].h, (grid[node_pos[0]][node_pos[1]].x, grid[node_pos[0]][node_pos[1]].y)))
                    removalCell = grid[node_pos[0]][node_pos[1]]
                    open_list.remove(removalCell)
                    removalCell.g = curr_cell.g+1
                    removalCell.set_fvalAdap()
                    heapq.heappush(open_list,removalCell)                                
                    parent_dict[(node_pos[0],node_pos[1])] = curr_cell # updating the parent dictionary
                    #parent_list.append(curr_cell)
                    #grid[node_pos[0]][node_pos[1]].pointer = curr_cell
                    #print(grid[node_pos[0]][node_pos[1]].pointer)
            else:
                # If the child is not visited update its g value, set its h and f value and add it to the 
                # children list after updating its parent.
                grid[node_pos[0]][node_pos[1]].g = curr_cell.g + 1
                grid[node_pos[0]][node_pos[1]].set_fvalAdap()
                children.append(grid[node_pos[0]][node_pos[1]])
                parent_dict[(node_pos[0],node_pos[1])] = curr_cell
                #parent_list.append(curr_cell)
                #grid[node_pos[0]][node_pos[1]].pointer = curr_cell
        if(children == []): # If no valid child continue
            continue
        for child in children: # else if valid children exist add them to the open list and update the visited set.
            #open_list.add((child.f,child.h, (child.x, child.y)))
            heapq.heappush(open_list,child)                                        
            visited.add((child.x, child.y))
    return [], 0, 0

In [61]:
def repeated_forward_AdaptiveAstar(agent_env,real_mz,final_mz,start_cell_ind, goal_cell_ind):
    paths = []
    counter = 0
    while(1):
        counter += 1
        print(counter)
        path,c,f = AdaptiveAstar(agent_env,start_cell_ind,goal_cell_ind)
        if(path == []):
            print("I cannot reach the target")
            break
        else:
            p = final_path(path,agent_env,real_mz,final_mz,start_cell_ind, goal_cell_ind)
            if(p[-1] == goal_cell_ind):
                print("I reached the target")
                print("final")
                display_maze(final_mz,start_cell_ind,goal_cell_ind)
                print("-------")
                display_maze(agent_env,start_cell_ind,goal_cell_ind)
                return path
            else:
                start_cell_ind = p[-1]

In [104]:
Adagent_env = maze(101,101,0,0,100,100,0)
Adagent_env.DFS([],0,0,100,100)
display_maze(Adagent_env.mz,[0,0],[100,100])
Adfinal_mz = maze(101,101,0,0,100,100,1)
Adfinal_mz.DFS([],0,0,100,100)
display_maze(Adfinal_mz.mz,[0,0],[100,100])
path = repeated_forward_AdaptiveAstar(Adagent_env.mz,real_mz.mz,Adfinal_mz.mz,[0,0],[100,100])
print(path)

 s - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -